In [4]:
#to explore: the step make game lose = score should be lower? like: [-1,-1,0,0,1,0,0,0,0], next step should be 2, otherwise will lose for sure.
#to explore: add some noise into input/ output during training
#to explore: remove duplication on training data & score
#to explore: seperate game engine & cnn into seperate py file
import os.path
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch.optim as optim
import numpy as np
from shutil import copyfile
torch.set_default_tensor_type('torch.cuda.FloatTensor')

file_name_model_latest_version = 'model_latest_version.pt'
file_name_model_last_version = 'model_last_version.pt'
input_file_path = '../Data/training_data_input.csv'
score_file_path = '../Data/training_data_score.csv'
input_file_path_initiate = '../Data/training_data_input_initiate.csv'
score_file_path_initiate = '../Data/training_data_score_initiate.csv'
training_file_path = '../Data/training_data.csv'

#feel like this part can be improved in a way.
DQ_ratio = 0.80

def training_model():
    
    #1 means there is an existing model saved
    #0 means there is no model saved in folder
    
    if os.path.isfile(file_name_model_latest_version):
        indicator = 1
        input_data = data_input(input_file_path)
        score_data = data_input(score_file_path)
    else:
        indicator = 0
        input_data = data_input(input_file_path_initiate)
        score_data = data_input(score_file_path_initiate)
        #give some initial value is another option
        #input_data = torch.tensor([0,0,0,0,0,0,0,0,0], dtype=torch.float)
        #score_data = torch.tensor([0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11], dtype=torch.float)

    input = input_data
    target = score_data
    # create your optimizer
    net = Net()
    criterion = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr=0.1)
    if indicator == 1:
        state_dict_last_version = torch.load('model_latest_version.pt')['state_dict']
        net.load_state_dict(state_dict_last_version)
        #to explore function to load optimizer from existing model
        #optimizer.load_state_dict(torch.load('model_latest_version.pt')['optimizer'])
    # in your training loop:
    for i in range(1,3000):
        #to explore a batch job for training
        #to explore how to save best performance model during 6000 epochs
        optimizer.zero_grad()   # zero the gradient buffers
        output = net(input)
        loss = criterion(output, target)
        loss.backward()
        if i % 999 == 0:
            print(loss)
        optimizer.step()    # Does the update
    
    if indicator == 1:
        copyfile(file_name_model_latest_version, file_name_model_last_version)
    else:
        torch.save({'state_dict': net.state_dict(),'optimizer': optimizer.state_dict()}, file_name_model_last_version)
    torch.save({'state_dict': net.state_dict(),'optimizer': optimizer.state_dict()}, file_name_model_latest_version)

    
def data_input(file_path):
    #to explore: remove duplicatioin - if both input and score are same, then remove duplication
    data_original = pd.read_csv(file_path)
    #data_original = data_original.drop_duplicates(subset=None, keep='first', inplace=False)
    torch_tensor = torch.tensor(data_original.values, dtype=torch.float)
    #print(torch_tensor)
    return torch_tensor


def main():
    tie = 0
    win = 0
    loss = 0
    
    #load model
    model_last_version = Net()
    state_dict_last_version = torch.load(file_name_model_last_version)['state_dict']
    model_last_version.load_state_dict(state_dict_last_version)
    
    model_latest_version = Net()
    state_dict_latest_version = torch.load(file_name_model_latest_version)['state_dict']
    model_latest_version.load_state_dict(state_dict_latest_version)
    
    training_data = data_input(training_file_path)
    #print(training_data)
    
    #initiallization
    #data_original = [1,0,0,-1,0,0,1,-1,0]
    #data_original = [0,1,0,0,0,0,-1,0,0]
    #data_original = [-1,1,0,1,1,-1,-1,-1,1]
    #data_original = [-1,1,1,1,1,-1,-1,-1,1]
    #data_original = [0,0,0,0,0,0,0,0,0]
    #torch_tensor = torch.tensor(data_original, dtype=torch.float)
    #print('torch_tensor 1st')
    #print(torch_tensor)
    
    for x in range(0,82):
        torch_tensor = training_data[x].clone()
        for j in range(0,9):
            if j == 0:
                ###########another possible scenario: if input without equal occurance of 1 & -1 - pending for coding
                game_a = game_agent(torch_tensor,0)
                if game_a.verify_result() == True:
                    print('not valid input')
                    break
                elif 0 not in torch_tensor:
                    print('input tie & with 9 value')
                    break
                else:
                    #initial
                    input_status = []
                    next_action_taken = []
                    score = []
                    #score = torch.zeros(9)
            torch_tensor_saved = torch_tensor.clone()
            input_status.append(torch_tensor_saved.cpu().numpy())
            #predict
            if j % 2 == 0:
                next_vision = model_latest_version(torch_tensor)
            else:
                next_vision = model_last_version(torch_tensor)

            #print(next_vision)
            current_score = torch.zeros(9)
            #comprehance predict result to make impossible option as '-1'
            for k in range(0,9):
                if torch_tensor[k] != 0:
                    next_vision[k] = -1.1
                    current_score[k] = -1.1
            next_step = next_vision.argmax()
            #save next action

            next_action_taken.append(next_step.cpu().numpy())
            score.append(current_score.cpu().numpy())
            #generate result
            torch_tensor[next_step] = 1
            game_a = game_agent(torch_tensor,0)
            # varify status of result
            if game_a.verify_result() == True:
                #print(game_a.verify_result())
                #calculate DQ
                if j % 2 == 0:
                    win += 1
                else:
                    loss += 1
                #print('finished')
                ############ here as an enhancement on model, it could encourage model for a faster win strategy
                current_final_score = 1
                #update score
                for k in range(0, len(next_action_taken)):
                    score[-(k+1)][next_action_taken[-(k+1)]] = ((-1 * DQ_ratio)**(k))

                #####################break for what?
                break
            else:
                if 0 not in torch_tensor:
                    #print('tie')
                    tie = tie + 1
                    break
                else:
                    #generate next input
                    torch_tensor = torch_tensor * -1
        input_status_df = pd.DataFrame(input_status)
        input_status_df.to_csv('../Data/training_data_input.csv', index=False, mode='a', header=False)
        next_action_taken_df = pd.DataFrame(next_action_taken)
        next_action_taken_df.to_csv('../Data/training_data_next_action_taken.csv', index=False, mode='a', header=False)
        score_df = pd.DataFrame(score)
        score_df.to_csv('../Data/training_data_score.csv', index=False, mode='a', header=False)
    print("tie" + str(tie))
    print("win" + str(win))
    print("loss" + str(loss))
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(9, 27)
        self.fc2 = nn.Linear(27, 81)
        self.dropout2 = nn.Dropout(p=0.8, inplace=False)
        self.fc3 = nn.Linear(81, 27)
        self.dropout3 = nn.Dropout(p=0.8, inplace=False)
        self.fc4 = nn.Linear(27, 18)
        self.dropout4 = nn.Dropout(p=0.8, inplace=False)
        self.fc5 = nn.Linear(18, 9)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


class game_agent:
    def __init__(self, input_string, status):
        self.input_string = input_string
        self.status = status
        self.verify_Matrix = [[0,1,2],[3,4,5],[6,7,8],[0,3,6],[1,4,7],[2,5,8],[0,4,8],[2,4,6]]
        self.verify_Matrix = np.array(self.verify_Matrix)
        #print(self.verify_Matrix.argmax())
    def restart(self):
        print(self.input_string)
        return('dsff')
    def next_action(self):
        print('next action')
    def result(self):
        if self.input_string[1,1] == 1:
            return True
        else:
            return False
    def verify_result(self):
        status = False
        for i in range(0,8):
            a = self.input_string[self.verify_Matrix[i,0]]
            b = self.input_string[self.verify_Matrix[i,1]]
            c = self.input_string[self.verify_Matrix[i,2]]
            #print(a,b,c)
            if verify_3_same_value(a,b,c) == True:
                #print(1)
                #print(a,b,c)
                status = True
                return status
                break
        return status
def verify_3_same_value(a,b,c):
    if (a == b) & (b == c) & (a != 0):
        return True
    else:
        return False
    
if __name__ == "__main__":
    button = 1
    if button == 1:
        for loops in range(0,5):
            training_model()
            main()
    else:
        main()

tensor(0.0362, grad_fn=<MseLossBackward>)
tensor(0.0358, grad_fn=<MseLossBackward>)
tensor(0.0354, grad_fn=<MseLossBackward>)
tie3
win57
loss22
tensor(0.0358, grad_fn=<MseLossBackward>)
tensor(0.0354, grad_fn=<MseLossBackward>)
tensor(0.0351, grad_fn=<MseLossBackward>)
tie1
win65
loss16
tensor(0.0356, grad_fn=<MseLossBackward>)
tensor(0.0352, grad_fn=<MseLossBackward>)
tensor(0.0349, grad_fn=<MseLossBackward>)
tie2
win61
loss19
tensor(0.0354, grad_fn=<MseLossBackward>)
tensor(0.0350, grad_fn=<MseLossBackward>)
tensor(0.0346, grad_fn=<MseLossBackward>)
tie2
win64
loss16
tensor(0.0348, grad_fn=<MseLossBackward>)
tensor(0.0346, grad_fn=<MseLossBackward>)
tensor(0.0343, grad_fn=<MseLossBackward>)
tie6
win62
loss14


In [19]:
#to explore: a UI for game - it can be a excel or something.

import os.path
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch.optim as optim
import numpy as np
from shutil import copyfile
torch.set_default_tensor_type('torch.cuda.FloatTensor')

file_name_model_latest_version = 'model_latest_version.pt'
file_name_model_last_version = 'model_last_version.pt'
input_file_path = '../Data/training_data_input.csv'
score_file_path = '../Data/training_data_score.csv'
input_file_path_initiate = '../Data/training_data_input_initiate.csv'
score_file_path_initiate = '../Data/training_data_score_initiate.csv'
training_file_path = '../Data/training_data.csv'

#feel like this part can be improved in a way.
DQ_ratio = 0.80

    
def data_input(file_path):
    #to explore: remove duplicatioin - if both input and score are same, then remove duplication
    data_original = pd.read_csv(file_path)
    #data_original = data_original.drop_duplicates(subset=None, keep='first', inplace=False)
    torch_tensor = torch.tensor(data_original.values, dtype=torch.float)
    #print(torch_tensor)
    return torch_tensor


def main():
    tie = 0
    win = 0
    loss = 0
    
    #load model
    model_last_version = Net()
    state_dict_last_version = torch.load(file_name_model_last_version)['state_dict']
    model_last_version.load_state_dict(state_dict_last_version)
    
    model_latest_version = Net()
    state_dict_latest_version = torch.load(file_name_model_latest_version)['state_dict']
    model_latest_version.load_state_dict(state_dict_latest_version)
    
    training_data = data_input(training_file_path)
    #print(training_data)
    
    #initiallization
    #data_original = [1,0,0,-1,0,0,1,-1,0]
    #data_original = [0,1,0,0,0,0,-1,0,0]
    #data_original = [-1,1,0,1,1,-1,-1,-1,1]
    #data_original = [-1,1,1,1,1,-1,-1,-1,1]
    data_original = [-1,-1,0,0,1,0,0,0,0]
    torch_tensor = torch.tensor(data_original, dtype=torch.float)
    #print('torch_tensor 1st')
    #print(torch_tensor)
    

    ###########another possible scenario: if input without equal occurance of 1 & -1 - pending for coding
    game_a = game_agent(torch_tensor,0)
    if game_a.verify_result() == True:
        print('not valid input')
    elif 0 not in torch_tensor:
        print('input tie & with 9 value')
    else:
        #initial
        input_status = []
        next_action_taken = []
        score = []
        #score = torch.zeros(9)
    torch_tensor_saved = torch_tensor.clone()
    input_status.append(torch_tensor_saved.cpu().numpy())
    #predict

    next_vision = model_latest_version(torch_tensor)

    print('next_vision')
    print(next_vision)
    current_score = torch.zeros(9)
    #comprehance predict result to make impossible option as '-1'
    for k in range(0,9):
        if torch_tensor[k] != 0:
            next_vision[k] = -1.1
            current_score[k] = -1.1
    next_step = next_vision.argmax()
    print('next_step')
    print(next_step)
    #save next action

    next_action_taken.append(next_step.cpu().numpy())
    score.append(current_score.cpu().numpy())
    #generate result
    torch_tensor[next_step] = 1
    game_a = game_agent(torch_tensor,0)
    # varify status of result
    if game_a.verify_result() == True:
        #print(game_a.verify_result())
        #calculate DQ
        win += 1
        #print('finished')
        ############ here as an enhancement on model, it could encourage model for a faster win strategy
        current_final_score = 1
        #update score
        for k in range(0, len(next_action_taken)):
            score[-(k+1)][next_action_taken[-(k+1)]] = ((-1 * DQ_ratio)**(k))
    else:
        if 0 not in torch_tensor:
            #print('tie')
            tie = tie + 1
        else:
            #generate next input
            torch_tensor = torch_tensor * -1

    print("tie" + str(tie))
    print("win" + str(win))
    print("loss" + str(loss))
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(9, 27)
        self.fc2 = nn.Linear(27, 81)
        self.dropout2 = nn.Dropout(p=0.8, inplace=False)
        self.fc3 = nn.Linear(81, 27)
        self.dropout3 = nn.Dropout(p=0.8, inplace=False)
        self.fc4 = nn.Linear(27, 18)
        self.dropout4 = nn.Dropout(p=0.8, inplace=False)
        self.fc5 = nn.Linear(18, 9)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


class game_agent:
    def __init__(self, input_string, status):
        self.input_string = input_string
        self.status = status
        self.verify_Matrix = [[0,1,2],[3,4,5],[6,7,8],[0,3,6],[1,4,7],[2,5,8],[0,4,8],[2,4,6]]
        self.verify_Matrix = np.array(self.verify_Matrix)
        #print(self.verify_Matrix.argmax())
    def restart(self):
        print(self.input_string)
        return('dsff')
    def next_action(self):
        print('next action')
    def result(self):
        if self.input_string[1,1] == 1:
            return True
        else:
            return False
    def verify_result(self):
        status = False
        for i in range(0,8):
            a = self.input_string[self.verify_Matrix[i,0]]
            b = self.input_string[self.verify_Matrix[i,1]]
            c = self.input_string[self.verify_Matrix[i,2]]
            #print(a,b,c)
            if verify_3_same_value(a,b,c) == True:
                #print(1)
                #print(a,b,c)
                status = True
                return status
                break
        return status
def verify_3_same_value(a,b,c):
    if (a == b) & (b == c) & (a != 0):
        return True
    else:
        return False
    
if __name__ == "__main__":
    button = 0
    if button == 1:
        for loops in range(0,5):
            training_model()
            main()
    else:
        main()

next_vision
tensor([-1.1361, -1.1056, -0.0312, -0.0265, -1.2079,  0.0170,  0.0865,  0.0342,
        -0.0761], grad_fn=<AddBackward0>)
next_step
tensor(6)
tie0
win0
loss0
